In [31]:
import pandas as pd
import numpy as np
import datetime
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import defaultdict
import re



In [32]:
df = pd.read_csv('AccidentesAviones.csv')

In [33]:
df.shape

(5008, 18)

In [34]:
df.columns

Index(['Unnamed: 0', 'fecha', 'HORA declarada', 'Ruta', 'OperadOR',
       'flight_no', 'route', 'ac_type', 'registration', 'cn_ln', 'all_aboard',
       'PASAJEROS A BORDO', 'crew_aboard', 'cantidad de fallecidos',
       'passenger_fatalities', 'crew_fatalities', 'ground', 'summary'],
      dtype='object')

In [35]:
df.dtypes

Unnamed: 0                 int64
fecha                     object
HORA declarada            object
Ruta                      object
OperadOR                  object
flight_no                 object
route                     object
ac_type                   object
registration              object
cn_ln                     object
all_aboard                object
PASAJEROS A BORDO         object
crew_aboard               object
cantidad de fallecidos    object
passenger_fatalities      object
crew_fatalities           object
ground                    object
summary                   object
dtype: object

In [36]:
df.head()

,Unnamed: 0,fecha,HORA declarada,Ruta,OperadOR,flight_no,route,ac_type,registration,cn_ln,all_aboard,PASAJEROS A BORDO,crew_aboard,cantidad de fallecidos,passenger_fatalities,crew_fatalities,ground,summary
0,0,"September 17, 1908",1718,"Fort Myer, Virginia",Military - U.S. Army,?,Demonstration,Wright Flyer III,?,1,2,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly..."
1,1,"September 07, 1909",?,"Juvisy-sur-Orge, France",?,?,Air show,Wright Byplane,SC1,?,1,0,1,1,0,0,0,Eugene Lefebvre was the first pilot to ever be...
2,2,"July 12, 1912",0630,"Atlantic City, New Jersey",Military - U.S. Navy,?,Test flight,Dirigible,?,?,5,0,5,5,0,5,0,First U.S. dirigible Akron exploded just offsh...
3,3,"August 06, 1913",?,"Victoria, British Columbia, Canada",Private,?,?,Curtiss seaplane,?,?,1,0,1,1,0,1,0,The first fatal airplane accident in Canada oc...
4,4,"September 09, 1913",1830,Over the North Sea,Military - German Navy,?,?,Zeppelin L-1 (airship),?,?,20,?,?,14,?,?,0,The airship flew into a thunderstorm and encou...


In [37]:
# rename columns
df.columns=['delete','date', 'time', 'location', 'operator', 'flight', 'route', 'type', 
                'registration', 'cn/ln', 'total_aboard', 'passengers_aboard', 
                'crew_aboard', 'total_fatalities', 'passengers_fatalities', 
                'crew_fatalities', 'ground', 'summary']

In [38]:
df.head()

,delete,date,time,location,operator,flight,route,type,registration,cn/ln,total_aboard,passengers_aboard,crew_aboard,total_fatalities,passengers_fatalities,crew_fatalities,ground,summary
0,0,"September 17, 1908",1718,"Fort Myer, Virginia",Military - U.S. Army,?,Demonstration,Wright Flyer III,?,1,2,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly..."
1,1,"September 07, 1909",?,"Juvisy-sur-Orge, France",?,?,Air show,Wright Byplane,SC1,?,1,0,1,1,0,0,0,Eugene Lefebvre was the first pilot to ever be...
2,2,"July 12, 1912",0630,"Atlantic City, New Jersey",Military - U.S. Navy,?,Test flight,Dirigible,?,?,5,0,5,5,0,5,0,First U.S. dirigible Akron exploded just offsh...
3,3,"August 06, 1913",?,"Victoria, British Columbia, Canada",Private,?,?,Curtiss seaplane,?,?,1,0,1,1,0,1,0,The first fatal airplane accident in Canada oc...
4,4,"September 09, 1913",1830,Over the North Sea,Military - German Navy,?,?,Zeppelin L-1 (airship),?,?,20,?,?,14,?,?,0,The airship flew into a thunderstorm and encou...


DATA DICTIONARY:

date: Date of accident, in the format - January 01, 2001
time: Local time, in 24 hr. format, unless specified otherwise (to be dropped after combining with date)
operator: Airline or aircraft operator
flight: Flight number assigned by the aircraft operator (to be dropped)
route: Complete or partial route flown prior to the accident
type: Aircraft type
registration: ICAO registration of the aircraft (to be dropped)
cn / ln: Construction or serial number / Line or fuselage number (to be dropped)
passengers_aboard: Total aboard (passengers)
crew_aboard: Total aboard (crew)
total_fatalities: Total fatalities aboard (passengers / crew)
passenger_fatalities: Total aboard passenger fatalities
crew_fatalities: Total aboard crew fatalities
ground: Total deaths on ground
summary: Brief description of the accident and cause, if known (may be droppable)


In [39]:
df.dtypes

delete                    int64
date                     object
time                     object
location                 object
operator                 object
flight                   object
route                    object
type                     object
registration             object
cn/ln                    object
total_aboard             object
passengers_aboard        object
crew_aboard              object
total_fatalities         object
passengers_fatalities    object
crew_fatalities          object
ground                   object
summary                  object
dtype: object

In [40]:
#drop the useless columns
df.drop('delete', axis=1, inplace=True)
df.drop(columns='flight', inplace=True)
df.drop(columns='cn/ln', inplace=True)
df.drop(columns='registration', inplace=True)


In [41]:
#turn date to date time
df["date"] = pd.to_datetime(df["date"], format='%B %d, %Y')
#replace incorrect data from the time column and format it to be the first 2 numbers:second 2 numbers, then set it as datetime with hour:minute format and fill missing values with 00:00
df['time'] = df['time'].str.replace('[^0-9]+', '', regex=True)
df['time'] = df['time'].str[:2] + ':' + df['time'].str[2:]
df['time'] = pd.to_datetime(df['time'], format='%H:%M', errors='coerce').dt.strftime('%H:%M')
df['time'].fillna('00:00', inplace=True)


In [42]:
# Combine 'date' and 'time' columns into a single datetime column and drop time column
df['date'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['time'].astype(str))
df.drop(columns='time', inplace=True) 

In [43]:
#counting remaining columns with ? as data
columns_with_question_mark = df.columns[df.eq('?').any()]
column_counts = df[columns_with_question_mark].apply(lambda x: x.value_counts()['?'])
column_counts_dict = column_counts.to_dict()
column_counts_dict

{'location': 5,
 'operator': 10,
 'route': 762,
 'type': 13,
 'total_aboard': 17,
 'passengers_aboard': 221,
 'crew_aboard': 219,
 'total_fatalities': 8,
 'passengers_fatalities': 235,
 'crew_fatalities': 235,
 'ground': 44,
 'summary': 59}

In [44]:
#brute force is the way. aint got no time for fancy functions.
columns_to_fill = {
    "location": "Unknown",
    "operator": "Unknown",
    "route": "Unknown",
    "type": "Unknown",
    "summary": "Unknown",
    "total_aboard": "0",
    "passengers_aboard": "0",
    "crew_aboard":"0",
    "total_fatalities":"0",
    "passengers_fatalities":"0",
    "crew_fatalities":"0",
    "ground":"0"
    }

for column, value in columns_to_fill.items():
    df[column] = df[column].replace('?', value)

In [45]:
df.head()

,date,location,operator,route,type,total_aboard,passengers_aboard,crew_aboard,total_fatalities,passengers_fatalities,crew_fatalities,ground,summary
0,1908-09-17 17:18:00,"Fort Myer, Virginia",Military - U.S. Army,Demonstration,Wright Flyer III,2,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly..."
1,1909-09-07 00:00:00,"Juvisy-sur-Orge, France",Unknown,Air show,Wright Byplane,1,0,1,1,0,0,0,Eugene Lefebvre was the first pilot to ever be...
2,1912-07-12 06:30:00,"Atlantic City, New Jersey",Military - U.S. Navy,Test flight,Dirigible,5,0,5,5,0,5,0,First U.S. dirigible Akron exploded just offsh...
3,1913-08-06 00:00:00,"Victoria, British Columbia, Canada",Private,Unknown,Curtiss seaplane,1,0,1,1,0,1,0,The first fatal airplane accident in Canada oc...
4,1913-09-09 18:30:00,Over the North Sea,Military - German Navy,Unknown,Zeppelin L-1 (airship),20,0,0,14,0,0,0,The airship flew into a thunderstorm and encou...


In [46]:
df.dtypes

date                     datetime64[ns]
location                         object
operator                         object
route                            object
type                             object
total_aboard                     object
passengers_aboard                object
crew_aboard                      object
total_fatalities                 object
passengers_fatalities            object
crew_fatalities                  object
ground                           object
summary                          object
dtype: object

In [47]:
#change columns to the appropriate data type
df['total_aboard'] = pd.to_numeric(df['total_aboard'], errors='coerce')
df['passengers_aboard'] = pd.to_numeric(df['passengers_aboard'], errors='coerce')
df['crew_aboard'] = pd.to_numeric(df['crew_aboard'], errors='coerce')
df['total_fatalities'] = pd.to_numeric(df['total_fatalities'], errors='coerce')
df['passengers_fatalities'] = pd.to_numeric(df['passengers_fatalities'], errors='coerce')
df['crew_fatalities'] = pd.to_numeric(df['crew_fatalities'], errors='coerce')
df['ground'] = pd.to_numeric(df['ground'], errors='coerce')

In [48]:
df.dtypes

date                     datetime64[ns]
location                         object
operator                         object
route                            object
type                             object
total_aboard                      int64
passengers_aboard                 int64
crew_aboard                       int64
total_fatalities                  int64
passengers_fatalities             int64
crew_fatalities                   int64
ground                            int64
summary                          object
dtype: object

i wanted to see what the word count on summary was, to see if i could extract something of value from here.
using the code from a classmate, who posted it on slack. 
this gave me the idea of filtering the possible causes of the crash according to certain keywords.

In [49]:
summary_data = df['summary'].tolist()
stop_words = set(stopwords.words('english'))
word_frequencies = defaultdict(int)
for summary in summary_data:
    tokens = word_tokenize(summary)
    filtered_words = [word.lower() for word in tokens if word.isalpha(
    ) and word.lower() not in stop_words and len(word) > 2]
    for word in filtered_words:
        word_frequencies[word] += 1
sorted_word_frequencies = sorted(
    word_frequencies.items(), key=lambda x: x[1], reverse=True)
for word, frequency in sorted_word_frequencies:
    print(f"{word}: {frequency}")

crashed: 3386
aircraft: 2640
plane: 2254
crew: 1132
flight: 1120
pilot: 1116
runway: 1035
engine: 1003
approach: 979
failure: 854
airport: 788
landing: 787
altitude: 645
weather: 637
taking: 603
control: 592
land: 583
conditions: 546
takeoff: 540
ground: 538
cargo: 535
struck: 527
fire: 521
mountain: 513
miles: 496
attempting: 486
route: 451
lost: 428
poor: 414
accident: 401
left: 388
due: 382
shortly: 376
killed: 372
right: 358
feet: 357
loss: 339
flying: 338
fuel: 334
fog: 333
failed: 333
short: 318
two: 315
error: 314
air: 313
hit: 311
captain: 311
low: 307
one: 305
emergency: 302
wing: 302
area: 289
terrain: 283
caused: 282
minutes: 277
sea: 267
power: 266
descent: 243
trees: 237
heavy: 237
unknown: 232
helicopter: 228
cause: 226
high: 218
visibility: 217
stalled: 216
crash: 214
field: 202
airplane: 200
flames: 200
rain: 200
broke: 199
turn: 196
engines: 193
made: 192
found: 187
time: 186
near: 185
speed: 183
flew: 180
resulted: 176
pilots: 174
system: 172
burned: 169
went: 167
abo

In [50]:
def category_summary(searchIn, word_frequencies):
    weather_keywords = ['lightning', 'storm', 'rain', 'fog', 'foggy', 'wind', 'winds', 'weather', 'blizzard',
                        'cloud', 'thunderstorm', 'thunderstorms', 'rainstorm', 'visibility', 'snowstorm', 'clouds', 'poor']
    derribo_keywords = ['shot', 'missile', 'shell', 'mortar', 'machine gun']
    despegue_keywords = ['taking', 'takeoff', 'take', 'off', 'taking off', 'taken off']
    falla_keywords = ['controls', 'fuel', 'failure', 'fail', 'failed', 'malfunction', 'malfunctioning', 'malfunctions', 'separated', 'broke', 'broke-up']
    aterrizaje_keywords = ['approach', 'landing', 'land', 'to land', 'short of the runway', 'overshot']

    for word in searchIn.split():
        if any(re.search(r'\b' + re.escape(word) + r'\b', searchIn, re.IGNORECASE) for word in weather_keywords):
            return 'clima'
        elif any(re.search(r'\b' + re.escape(word) + r'\b', searchIn, re.IGNORECASE) for word in derribo_keywords):
            return 'derribo'
        elif any(re.search(r'\b' + re.escape(word) + r'\b', searchIn, re.IGNORECASE) for word in despegue_keywords):
            return 'despegue'
        elif any(re.search(r'\b' + re.escape(word) + r'\b', searchIn, re.IGNORECASE) for word in falla_keywords):
            return 'falla'
        elif any(re.search(r'\b' + re.escape(word) + r'\b', searchIn, re.IGNORECASE) for word in aterrizaje_keywords):
            return 'aterrizaje'

    return 'indeterminado'


In [51]:
df['category'] = df['summary'].apply(lambda row: category_summary(row, word_frequencies))


In [52]:
df.head()

,date,location,operator,route,type,total_aboard,passengers_aboard,crew_aboard,total_fatalities,passengers_fatalities,crew_fatalities,ground,summary,category
0,1908-09-17 17:18:00,"Fort Myer, Virginia",Military - U.S. Army,Demonstration,Wright Flyer III,2,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly...",falla
1,1909-09-07 00:00:00,"Juvisy-sur-Orge, France",Unknown,Air show,Wright Byplane,1,0,1,1,0,0,0,Eugene Lefebvre was the first pilot to ever be...,falla
2,1912-07-12 06:30:00,"Atlantic City, New Jersey",Military - U.S. Navy,Test flight,Dirigible,5,0,5,5,0,5,0,First U.S. dirigible Akron exploded just offsh...,indeterminado
3,1913-08-06 00:00:00,"Victoria, British Columbia, Canada",Private,Unknown,Curtiss seaplane,1,0,1,1,0,1,0,The first fatal airplane accident in Canada oc...,indeterminado
4,1913-09-09 18:30:00,Over the North Sea,Military - German Navy,Unknown,Zeppelin L-1 (airship),20,0,0,14,0,0,0,The airship flew into a thunderstorm and encou...,clima


In [53]:
summary_data = df['operator'].tolist()
stop_words = set(stopwords.words('english'))
word_frequencies = defaultdict(int)
for summary in summary_data:
    tokens = word_tokenize(summary)
    filtered_words = [word.lower() for word in tokens if word.isalpha(
    ) and word.lower() not in stop_words and len(word) > 2]
    for word in filtered_words:
        word_frequencies[word] += 1
sorted_word_frequencies = sorted(
    word_frequencies.items(), key=lambda x: x[1], reverse=True)
for word, frequency in sorted_word_frequencies:
    print(f"{word}: {frequency}")

air: 1481
airlines: 839
military: 776
force: 557
airways: 453
aeroflot: 265
lines: 183
aviation: 144
royal: 130
american: 124
army: 113
transport: 99
service: 94
national: 93
british: 85
navy: 78
private: 78
france: 78
united: 78
china: 77
lufthansa: 74
trans: 72
world: 69
deutsche: 67
services: 65
indian: 61
taxi: 60
pan: 59
international: 55
pacific: 53
corporation: 52
charter: 51
inc: 48
western: 47
forces: 47
express: 45
aero: 45
philippine: 45
eastern: 43
cargo: 42
colombia: 42
continental: 40
lineas: 39
aerial: 38
klm: 37
dutch: 36
orient: 36
mail: 35
overseas: 35
south: 35
helicopters: 34
northwest: 33
union: 30
aerolineas: 30
russian: 30
avianca: 29
afghan: 29
new: 28
central: 28
vietnam: 27
flying: 26
pakistan: 26
indonesia: 26
aérea: 25
aeropostale: 25
imperial: 25
transportes: 25
sabena: 24
aviacion: 24
aéreas: 24
australian: 23
aereo: 23
helicopter: 22
german: 21
company: 21
vasp: 21
corps: 21
european: 21
alaska: 21
fuerza: 20
african: 20
tam: 20
del: 20
garuda: 20
merpati

we can do the same with the operator column to further filter military and civilian flights.

In [54]:
def operator_summary(searchIn, word_frequencies):
    for palabra in searchIn.split():
        if palabra.lower() in ['military', 'navy', 'force', 'air forces']:
            return 'Militar'

    filtered_words = [
        word.lower()
        for word in word_tokenize(searchIn)
        if word.isalpha() and word.lower() not in stop_words and len(word) > 2
    ]
    for word in filtered_words:
        word_frequencies[word] += 1
    
    return 'Civil'

In [55]:
summary_data = df['operator'].tolist()
stop_words = set(stopwords.words('english'))
word_frequencies = defaultdict(int)

df['flight_type'] = df['operator'].apply(lambda row: operator_summary(row, word_frequencies))


In [56]:
df["survival_rate"] = 100 * (df["total_aboard"] - df["total_fatalities"]) / df["total_aboard"]

In [58]:
df.head()

,date,location,operator,route,type,total_aboard,passengers_aboard,crew_aboard,total_fatalities,passengers_fatalities,crew_fatalities,ground,summary,category,flight_type,survival_rate
0,1908-09-17 17:18:00,"Fort Myer, Virginia",Military - U.S. Army,Demonstration,Wright Flyer III,2,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly...",falla,Militar,50.0
1,1909-09-07 00:00:00,"Juvisy-sur-Orge, France",Unknown,Air show,Wright Byplane,1,0,1,1,0,0,0,Eugene Lefebvre was the first pilot to ever be...,falla,Civil,0.0
2,1912-07-12 06:30:00,"Atlantic City, New Jersey",Military - U.S. Navy,Test flight,Dirigible,5,0,5,5,0,5,0,First U.S. dirigible Akron exploded just offsh...,indeterminado,Militar,0.0
3,1913-08-06 00:00:00,"Victoria, British Columbia, Canada",Private,Unknown,Curtiss seaplane,1,0,1,1,0,1,0,The first fatal airplane accident in Canada oc...,indeterminado,Civil,0.0
4,1913-09-09 18:30:00,Over the North Sea,Military - German Navy,Unknown,Zeppelin L-1 (airship),20,0,0,14,0,0,0,The airship flew into a thunderstorm and encou...,clima,Militar,30.0


In [57]:
# COMMENT THIS SO IT DOESNT DO ANYTHING WEIRD EVERY TIME IT RUNS EVERYTHING.
df.to_csv('clean_accidents.csv', index=False)